# Tutorial - Converter View para Materialized View

Este material demonstra de maneira simplificada **como uma view comum pode ser transformada em uma view materializada por meio de um script**. No exemplo aplicado, basicamente tenta-se criar uma view materializada utilizando o mesmo conteúdo de SQL já existente na view comum - apenas incrementando a instrução `CREATE MATERIALIZED VIEW`.

Além de demonstrar a possibilidade de conversão, **objetiva-se também incentivar a evolução de um algorítimo** que proporcione um método de conversão mais amplo, **adaptando incompatibilidades entre uma view comum e uma view materializada sem afetar a regra de negócio** - por exemplo a remoção de cláusulas `ORDER BY` - que não são permitidas numa view materializada.

Os pré-requisitos para criar uma view materializada bem como suas limitações, podem ser verificados aqui https://cloud.google.com/bigquery/docs/materialized-views

- Neste exemplo, as views convencionais que puderem ser convertidas em views materializadas, serão criadas com o mesmo nome, no mesmo projeto e dataset da view comum, porém com o prefixo `MVW_`

- O resulado final das tentativas de convesão (bem sucedidas ou não), serão exportados para o arquivo `resultado_conversao.json`

Antes de executar este arquivo, é necessário **executar os passos de 1 a 6 do aquivo `view-streaming.ipynb`**
   
  
_Rubens Mussi Cury_     
🥇Google Cloud Certified   
rubensmussicury@gmail.com   

## Bibliotecas
* Conta de Serviço - https://cloud.google.com/docs/authentication/production
* BigQuery - https://cloud.google.com/bigquery/docs/how-to e https://google-cloud-python.readthedocs.io/en/0.32.0/bigquery/usage.html
* Exceções - https://google-cloud-python.readthedocs.io/en/0.32.0/core/exceptions.html

In [ ]:
import os
import json
from google.oauth2 import service_account
from google.cloud import bigquery
from google.api_core.exceptions import BadRequest, Conflict

## Autenticação
* https://cloud.google.com/iam/docs/creating-managing-service-account-keys?hl=pt-br
* Via `os.environ["GOOGLE_APPLICATION_CREDENTIALS"]`
* Via service_account.Credentials()

In [ ]:
# Caminho completo da chave de serviço
gcp_credential_file = "sua-chave-aqui.json"

# Autenticação na conta de serviço.
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = gcp_credential_file

# Utilize esta forma de credencial para autenticar diretamente pelo SDK.
# credentials = service_account.Credentials.from_service_account_file(gcp_credential_file)
# client = bigquery.Client(credentials=credentials)

## Definição do Ambiente
- Identificação do Projeto
- Listar Datasets
- Encontrar `VIEW`
- Tentar transformá-las em `MATERIALIZED_VIEW`

In [ ]:
# Instancia o Client do BQ.
bq_client = bigquery.Client()

# Define único global do projeto.
project_id = bq_client.project

In [ ]:
# Obtém a lista de datasets existentes no projeto.
dataset_list = list(bq_client.list_datasets())

dataset_ids = []
for dataset in dataset_list:
    dataset_ids.append(dataset.dataset_id)

## Realiza a Conversão
- Percorre um a um das objetos de cada dataset
- Verifica se é um `VIEW`
- Tenta converter para `MATERIALIZED_VIEW`
- Gera arquivo retorno das conversões

In [ ]:
# Resultado de saída.
output_json = {} 

# Percorre datasets.
for dataset_id in dataset_ids:
    
    # Obtém todos os objetos contidos no dataset.
    dataset_objects = bq_client.list_tables(dataset_id)
    
    # Percorre cada um dos objetos do dataset.
    for dataset_object in dataset_objects:

        # Determina o nome completo do objeto no formato project.dataset.objectId
        object_id = "{0}.{1}.{2}".format(dataset_object.project, dataset_object.dataset_id, dataset_object.table_id)
        
        # Verifica se o objeto é uma view comum.
        if dataset_object.table_type == "VIEW":
            
            # Busca e instancia objeto corrente.
            fetched_object = bq_client.get_table(object_id)
            
            # Define o nome que será utilizado para a view materializada no formato project.dataset.MVW_objectId
            mview_name = "{0}.{1}.MVW_{2}".format(dataset_object.project,
                                                  dataset_object.dataset_id,
                                                  dataset_object.table_id)
            
            # Define a sintaxe para criação de uma `MATERIALIED_VIEW`
            mview_sintax = "CREATE MATERIALIZED VIEW `{0}` AS \n {1}".format(mview_name, fetched_object.view_query)
            # Executa job no BigQuery com a sintaxe em SQL.
            executed_job = bq_client.query(mview_sintax)
            
            try:
                # Aguarda o término da execução do job.
                executed_job.result()
                
                opt_result = "Success"
                opt_error_type = ""
                opt_materialized_view = mview_name
                opt_message = "View convertida com sucesso"
            
            # Chamada inválida.
            except BadRequest as e:
                opt_result = "Error"
                opt_error_type = "BadRequest"
                opt_materialized_view = ""
                opt_message = "/n".join(e.args)
                
            # Objeto já existente.
            except BadRequest as e:
                opt_result = "Error"
                opt_error_type = "AlreadyExists"
                opt_materialized_view = ""
                opt_message = "/n".join(e.args)
            
            # Exceção genérica.
            except Exception as e:
                opt_result = "Error"
                opt_error_type = "CheckErrorMessage"
                opt_materialized_view = ""
                opt_message = "/n".join(e.args)
            
            job_id = executed_job.job_id
            output_json[job_id] = {}
            output_json[job_id]["result"] = opt_result 
            output_json[job_id]["error_type"] = opt_error_type 
            output_json[job_id]["regular_view"] = object_id
            output_json[job_id]["materialized_view"] = opt_materialized_view
            output_json[job_id]["message"] = opt_message

# Gera JSON com resultado das conversões.
with open('resultado_conversao.json', 'w') as outfile:
    json.dump(output_json, outfile, indent=4)